In [24]:
from pathlib import Path

In [23]:
#| code-fold: true
#| code-summary: The full contents of our Dockerfile

!cat db_context/Dockerfile

FROM postgis/postgis:15-3.3

You can define and configure the services, networks, volumes, and environment variables (or secrets) your system needs in the `docker-compose.yml` file. The `docker-compose.yml` file below only defines one service (named `postgis`) which will be provided by a container based on the image defined by our (above) `Dockerfile`. PostgreSQL databases listen to port number 5432 by default, but I already have a postgres database connected to port 5432 on my host machine, so I've used the **ports** section to indicate port 5432 inside the container should be connected to port 54321 on the host system. Docker containers are designed to be a replicable instance of an image, so when you shut down your application, your containers are removed and new ones are created next time you start it up. This is great for reproducibility (you always get a new, clean instance based on your image), but you don't want the data you ingest into your database to get wiped every time you shut down your system, so you can define a persistent **volume** that will live on in the host system. And when the postgres database intiailly starts up, the database also creates a superuser using the credentials we pass in as environment variables `POSTGRES_DB`, `POSTGRES_USER`, and `POSTGRES_PASSWORD`.

In [19]:
#| code-fold: true
#| code-summary: The contents of our docker-compose.yml

!cat docker-compose.yml

version: '3.9'

services:
  postgis:
    image: sandbox_postgis:15.3.3
    build:
      context: ./db_context
      dockerfile: Dockerfile
    ports:
      - 54321:5432
    environment:
      POSTGRES_DB: db_name
      POSTGRES_USER: db_username
      POSTGRES_PASSWORD: db_password
    volumes:
      - sandbox_postgis_data:/var/lib/postgresql/data

volumes:
  sandbox_postgis_data:


The first time you build the image defined by your Dockerfile, docker will read your Dockerfile(s), download all layers of the base image(s) (defined in lines starting with `FROM `), process each subsequent instruction into a layer, cache layers, and then bind the layers into an image. This is a template for creating containers.

The second time you run this command (assuming no changes have been made to the `docker-compose.yml` file, Dockerfile(s), or any other files the Dockerfile references), all layers will just be pulled from cache, producing much smaller output (like what's shown below).

In [2]:
!docker compose build

[+] Building 0.0s (0/2)                                                         
[+] Building 0.2s (2/3)                                                         
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [postgis internal] load metadata for docker.io/postgis/postgis:15-3.3  0.1s
[+] Building 0.3s (5/5) FINISHED                                                
 => [postgis internal] load build definition from Dockerfile               0.0s
 => => transferring dockerfile: 64B                                        0.0s
 => [postgis internal] load .dockerignore                                  0.0s
 => => transferring context: 2B                                            0.0s
 => [postgis internal] load metadata 

In [12]:
!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


`docker ps` shows running containers, and the `-f name=...` option allows us to filter to running containers with a name containing the entered string. Currently, "sandbox-postgis" isn't part of the name of any running container, so let's spin one up.

In [13]:
!docker compose up -d

[+] Running 2/0
 ✔ Network 015_docker_postgres_sandbox_default                Created      0.0s 
 ✔ Volume "015_docker_postgres_sandbox_sandbox_postgis_data"  Created      0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1            Starting     0.0s 
[+] Running 2/3
 ✔ Network 015_docker_postgres_sandbox_default                Created      0.0s 
 ✔ Volume "015_docker_postgres_sandbox_sandbox_postgis_data"  Created      0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1            Starting     0.1s 
[+] Running 2/3
 ✔ Network 015_docker_postgres_sandbox_default                Created      0.0s 
 ✔ Volume "015_docker_postgres_sandbox_sandbox_postgis_data"  Created      0.0s 
 ⠿ Container 015_docker_postgres_sandbox-postgis-1            Starting     0.2s 
[+] Running 2/3
 ✔ Network 015_docker_postgres_sandbox_default                Created      0.0s 
 ✔ Volume "015_docker_postgres_sandbox_sandbox_postgis_data"  Created      0.0s 
 ⠿ Container 015_docker_postgres_sandbox-post

In [21]:
!docker ps -f name=sandbox-postgis

CONTAINER ID   IMAGE                    COMMAND                  CREATED              STATUS              PORTS                                         NAMES
7d90dcd9c402   sandbox_postgis:15.3.3   "docker-entrypoint.s…"   About a minute ago   Up About a minute   0.0.0.0:54321->5432/tcp, :::54321->5432/tcp   015_docker_postgres_sandbox-postgis-1
